# Public Key Cryptography
Approximately all algorithms in this category are based on the three mathematical concepts
- Prime Facterisation of Integers
- Discrete Logrithms
- Elliptic Curves 

In [1]:
def mod_exp(base, exponent, modulus):
    result = 1 
    while exponent > 0:
        if exponent % 2 == 1:
            result = (result * base) % modulus
        exponent = exponent // 2
        base = (base * base) % modulus
    return result
        

In [4]:
mod_exp(10, 25, 7)

3

In [5]:
def diffie_hellman(p, g, private_key):
    public_key = mod_exp(g, private_key, p)
    return public_key

In [6]:
diffie_hellman(23, 9, 8)

13

In [7]:
def shared_secret(p, public_key, private_key):
    secret = mod_exp(public_key, private_key, p)
    return secret

In [9]:
shared_secret(23, 13, 8)

2

In [10]:
# shared prime number and primitives root modulo
p = 23 
g = 5

# Private Keys for the Alice and Bob
private_key_alice = 6
private_key_bob = 15

# Alice's and Bob's Public Key
public_key_alice = diffie_hellman(p, g, private_key_alice)
public_key_bob = diffie_hellman(p, g, private_key_bob)


In [11]:
shared_secret_alice = shared_secret(p, public_key_alice, private_key_alice)
shared_secret_bob = shared_secret(p, public_key_bob, private_key_bob)

In [12]:
print("Shared Secret of Alice : ", shared_secret_alice)

Shared Secret of Alice :  13


In [13]:
print("Shared secret of Bob : ", shared_secret_bob)

Shared secret of Bob :  20


In [4]:
# Elliptic Curve Parameters (secp256r1)
p = 0xFFFFFFFF00000001000000000000000000000000FFFFFFFFFFFFFFFFFFFFFFFF
a = 0xFFFFFFFF00000001000000000000000000000000FFFFFFFFFFFFFFFFFFFFFFFC
b = 0x5AC635D8AA3A93E7B3EBBD55769886BC651D06B0CC53B0F63BCE3C3E27D2604B
Gx = 0x6B17D1F2E12C4247F8BCE6E563A440F277037D812DEB33A0F4A13945D898C296
Gy = 0x4FE342E2FE1A7F9B8EE7EB4A7C0F9E162BCE33576B315ECECBB6406837BF51F5


In [5]:

# Point class for representing points on the elliptic curve
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

# Elliptic curve addition
def point_add(p, q):
    if p is None:
        return q
    if q is None:
        return p
    if p.x == q.x and p.y != q.y:
        return None
    if p != q:
        slope = (q.y - p.y) * pow(q.x - p.x, -1, p)
    else:
        slope = (3 * p.x * p.x + a) * pow(2 * p.y, -1, p)
    x = (slope * slope - p.x - q.x) % p
    y = (slope * (p.x - x) - p.y) % p
    return Point(x, y)


In [12]:
print(Point(23, 29))


In [15]:
def mod_inverse(a, m):
    # Extended Euclidean algorithm to compute the modular inverse
    if m == 0:
        raise ValueError("Modulus cannot be zero")
    if a < 0:
        a = a % m
    g, x, y = extended_gcd(a, m)
    if g != 1:
        raise ValueError("Modular inverse does not exist")
    return x % m

def extended_gcd(a, b):
    if a == 0:
        return b, 0, 1
    gcd, x1, y1 = extended_gcd(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    return gcd, x, y

In [16]:
# Point class for representing points on the elliptic curve
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

# Elliptic curve addition
def point_add(p, q):
    if p is None:
        return q
    if q is None:
        return p
    if p.x == q.x and p.y != q.y:
        return None
    if p != q:
        slope = (q.y - p.y) * mod_inverse(q.x - p.x, p)  # Use mod_inverse
    else:
        slope = (3 * p.x * p.x + a) * mod_inverse(2 * p.y, p)  # Use mod_inverse
    x = (slope * slope - p.x - q.x) % p
    y = (slope * (p.x - x) - p.y) % p
    return Point(x, y)

# Elliptic curve scalar multiplication with modular inverse
def point_mult(k, p):
    result = None
    addend = p
    while k:
        if k & 1:
            result = point_add(result, addend)
        addend = point_add(addend, addend)
        k >>= 1
    return result

# Generate private keys for Alice and Bob (random integers in the range [1, p-1])
alice_private_key = 123456789
bob_private_key = 987654321

# Compute public keys for Alice and Bob
alice_public_key = point_mult(alice_private_key, Point(Gx, Gy))
bob_public_key = point_mult(bob_private_key, Point(Gx, Gy))

# Compute shared secret for Alice and Bob
shared_secret_alice = point_mult(alice_private_key, bob_public_key).x
shared_secret_bob = point_mult(bob_private_key, alice_public_key).x

print("Shared Secret (Alice):", shared_secret_alice)
print("Shared Secret (Bob):", shared_secret_bob)

TypeError: unsupported operand type(s) for %: 'Point' and 'int'

In [6]:

# Elliptic curve scalar multiplication
def point_mult(k, p):
    result = None
    addend = p
    while k:
        if k & 1:
            result = point_add(result, addend)
        addend = point_add(addend, addend)
        k >>= 1
    return result


In [7]:

# Generate private keys for Alice and Bob (random integers in the range [1, p-1])
alice_private_key = 123456789
bob_private_key = 987654321

# Compute public keys for Alice and Bob
alice_public_key = point_mult(alice_private_key, Point(Gx, Gy))
bob_public_key = point_mult(bob_private_key, Point(Gx, Gy))

# Compute shared secret for Alice and Bob
shared_secret_alice = point_mult(alice_private_key, bob_public_key)
shared_secret_bob = point_mult(bob_private_key, alice_public_key)

print("Shared Secret (Alice):", shared_secret_alice.x)
print("Shared Secret (Bob):", shared_secret_bob.x)



TypeError: unsupported operand type(s) for ** or pow(): 'int', 'int', 'Point'

In [3]:
pow(34, 3)

39304